In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sktime.utils.plotting import plot_series, plot_windows
from feature_engine.timeseries.forecasting import WindowFeatures
from sktime.transformations.series.summarize import WindowSummarizer

### Data
We will work with the hourly electricity demand dataset. It is the electricity demand for the state of Victora in Australia from 2002 to the start of 2015. 


In [2]:
data = pd.read_csv('../../Datasets/victoria_electricity_demand.csv', 
                   usecols=["demand", "temperature", "date_time"], 
                   index_col='date_time', parse_dates=['date_time'])

# For this demo we will use a subset of the data
data = data.loc["2010":]

# plot
plot_series(data['demand'])
plt.xticks(rotation=30);

<img src='./plots/victoria-electricity-deman-2010-2015.png'>

## Lets look at the demand for year 2014 

In [30]:
data.loc["2014"].plot(y=['demand'], figsize=(15,5))

<img src='./plots/victoria-electricity-deman-2014.png'>

## Lets look at the temperature record of year 2014

In [32]:
data.loc["2014"].plot(y=['temperature'], figsize=(15,5))

<img src='./plots/victoria-temperature-2014.png'>

#### Temperature and Electricity demand is correlated 

In [38]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15,10))
data.loc["2014"].plot(y=['temperature'], title='Temperature reading 2014', ax=ax[0])
data.loc["2014"].plot(y=['demand'], title="Electricity demand 2014", ax=ax[1] )
plt.tight_layout()

<img src='./plots/victoria-electricity-demand-vs-temperature-2014.png'>

In [40]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15,10))
data.loc["2015-01-02"].plot(y=['temperature'], title='Temperature reading 2015-01-02', ax=ax[0])
data.loc["2015-01-02"].plot(y=['demand'], title="Electricity demand 2015-01-02", ax=ax[1] )
plt.tight_layout()

<img src='./plots/victoria-electricity-demand-vs-temperature-2015-01-02.png'>

In [6]:
df = data.copy()

#### We can see that there is seasonality on different time scales (e.g., daily, weekly, yearly). 
*  rolling window features can help capture information at these different time scales. 


We want to compute the rolling:
- mean
- standard deviation
- median absolute deviation

The median absolute deviation is not implemented natively in Pandas so we have to create it as a custom metric ourselves.

Let's create a function to compute the median absolute deviation which is defined as: $MAD = median(|x_i - median(x)|)$

In [14]:
def MAD(x):
    return np.median(np.abs(x-np.median(x)))

In [15]:
result = (df['demand'].rolling(window=24)
          .agg(func=['mean','std',MAD])
          .shift(periods=1, freq='H')
          .add_prefix('demand_window_24_'))

result.head()

,demand_window_24_mean,demand_window_24_std,demand_window_24_MAD
date_time,,,
2010-01-01 01:00:00,NaN,NaN,NaN
2010-01-01 02:00:00,NaN,NaN,NaN
2010-01-01 03:00:00,NaN,NaN,NaN
2010-01-01 04:00:00,NaN,NaN,NaN
2010-01-01 05:00:00,NaN,NaN,NaN


In [18]:
result.plot(y=['demand_window_24_mean', 'demand_window_24_std', 'demand_window_24_MAD'], subplots=True, figsize=(18, 8), title=['Measure of spread', 'Measure of volatility', 'Median absolute deviation']);

<img src='./plots/rolling-stats-pandas-24h-window.png'>

#### `min_periods` argument can be used to allow smaller window at the edges

In [19]:
result_pandas = (df['demand'].rolling(window=24, min_periods=1)
          .agg(func=['mean','std',MAD])
          .shift(periods=1, freq='H')
          .add_prefix('demand_window_24_'))

result_pandas.head()

,demand_window_24_mean,demand_window_24_std,demand_window_24_MAD
date_time,,,
2010-01-01 01:00:00,8314.448682,NaN,0.000000
2010-01-01 02:00:00,8290.817989,33.418847,23.630693
2010-01-01 03:00:00,7992.054807,518.012283,47.261386
2010-01-01 04:00:00,7732.052986,670.294667,459.960119
2010-01-01 05:00:00,7559.082315,697.542450,527.328810


In [59]:
result_pandas.plot(y=['demand_window_24_mean', 'demand_window_24_std', 'demand_window_24_MAD'], subplots=True, figsize=(18, 8), title=['Measure of spread', 'Measure of volatility', 'Median absolute deviation']);

<img src='./plots/rolling-stats-pandas-24h-window-with_min_periods.png'>

In [61]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(18,8), sharex=True, constrained_layout=True)
ax = ax.ravel()
cols = ['demand_window_24_std', 'demand_window_24_MAD']
title=['Standard Deviation', 'Median Absolute Deviation']

df.plot(y=['demand'], ax=ax[0], color='lightblue')
result_pandas.plot(y=['demand_window_24_mean'], color='salmon', title='Demand & 24hr window mean', ax=ax[0])
for i,frame in enumerate(ax[1:]):
    result_pandas.plot(y=[cols[i]], subplots=True, ax=frame, color='steelblue', title=[title[i]], alpha=0.8);

fig.suptitle('Electricity Demand | Rolling statistics', size=24)


<img src='./plots/Electricity-demand-rolling-stats-pandas-mean-std-mad.png'>

## Using Feature-engine

lets see how we can extract the same information using feature-engine

In [63]:
from feature_engine.timeseries.forecasting.window_features import WindowFeatures

transformer = WindowFeatures(variables=['demand', 'temperature'],
                             window=[24, 7*24, 365*24], functions=['mean', 'std'], freq="1H")

result_feature_engine = transformer.fit_transform(df)


In [78]:
cols = [c for c in result_feature_engine.columns if 'demand' in c]
ax = result_feature_engine.plot(y=cols, subplots=True, figsize=(15,10));

<img src='./plots/rolling-stats-feature-engine.png'>

#### We can see how different window sizes have smoothed over different seasonalities. This allows the model to make use of behaviours seen at different time scales (e.g., if the daily average of the demand is greater than the average of the past year then perhaps this can influence future values of our target variable).

In [84]:
result_feature_engine.filter(regex='demand_.*_mean', axis=1).plot(figsize=(16,8), title='Rolling stats');

<img src='./plots/feature-engine-rolling-mean.png'>

## Using Sktime



In [88]:
from sktime.transformations.series.summarize import WindowSummarizer

transformer = WindowSummarizer(
    target_cols=['demand'],
    lag_feature={
    "lag":[1,2,3],
    'mean':[[1, 24],[1, 7*24]],
    'std':[[1, 24],[1, 7*24]],
     MAD:[[1, 24],[1, 7*24]],
    }
)

result_sktime = transformer.fit_transform(df)

In [95]:
result_sktime.filter(regex='demand_mean.*').plot(figsize=(18,6), title="Rolling mean | Sktime")

<img src='./plots/rolling-stats-sktime.png'>